In [2]:
from loaders import *
from dataset import *
from episode import *
from collections import Counter
from torch import nn
from torch.autograd import Variable

import numpy as np
import torch
import torch.nn.functional as F
import json
import numpy as np
import matplotlib.pyplot as plt
import random

In [3]:
eps = load_sampler_from_config("config.yaml")

In [4]:
""" Some global variables """
_loader = Loader(502) # 500 + SOS + EOS
loader = MIDILoader(_loader)

use_cuda = torch.cuda.is_available()
# Is the tokenizer 1 indexed?
vocabulary_size = 16*128*2 + 32*16 + 100 + 1 # 4708 + 1
vocabulary_size = vocabulary_size + 2 # SOS (index 4709) and EOS (index 4710)
SOS_TOKEN = 4709
EOS_TOKEN = 4710

encoding_size = 500
one_hot_embeddings = np.eye(vocabulary_size)

In [5]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [6]:
class EncoderLSTM(nn.Module):
    # Your code goes here
    def __init__(self, input_size, hidden_size):
        super(EncoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size).double()
        if use_cuda:
            self.lstm = self.lstm.cuda()
        
    def forward(self, input, hidden_in):
        _, hidden_out = self.lstm(input, hidden_in) # encoder only outputs hidden
        return hidden_out
    
    def initHidden(self, hidden, batch_size):
        
        if hidden == None:
            result = Variable(torch.zeros(1, batch_size, self.hidden_size)).double()
            
            if use_cuda:
                result = result.cuda()
            return result
        
        else:
            return hidden

In [7]:
class DecoderLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(DecoderLSTM, self).__init__()
        self.hidden_size = hidden_size

        self.lstm = nn.LSTM(input_size, hidden_size).double()
        self.out = nn.Linear(hidden_size, output_size).double()
        if use_cuda:
            self.lstm = self.lstm.cuda()
            self.out = self.out.cuda()

    def forward(self, input, hidden):
        output = F.relu(input)
        output, hidden = self.lstm(output, hidden)
        output = self.out(output)
        output = output.squeeze()
        return output.unsqueeze(0), hidden

    def initHidden(self, batch_size):
        result = Variable(torch.zeros(1, batch_size, self.hidden_size)).double()
        if use_cuda:
            return result.cuda()
        else:
            return result

In [8]:
# The next two functions are part of some other deep learning frameworks, but PyTorch
# has not yet implemented them. We can find some commonly-used open source worked arounds
# after searching around a bit: https://gist.github.com/jihunchoi/f1434a77df9db1bb337417854b398df1.
def _sequence_mask(sequence_length, max_len=None):
    if max_len is None:
        max_len = sequence_length.data.max()
    batch_size = sequence_length.size(0)
    seq_range = torch.arange(0, max_len).long()
    seq_range_expand = seq_range.unsqueeze(0).expand(batch_size, max_len)
    seq_range_expand = Variable(seq_range_expand)
    if sequence_length.is_cuda:
        seq_range_expand = seq_range_expand.cuda()
    seq_length_expand = (sequence_length.unsqueeze(1)
                         .expand_as(seq_range_expand))
    return seq_range_expand < seq_length_expand


def compute_loss(logits, target, length):
    """
    Args:
        logits: A Variable containing a FloatTensor of size
            (batch, max_len, num_classes) which contains the
            unnormalized probability for each class.
        target: A Variable containing a LongTensor of size
            (batch, max_len) which contains the index of the true
            class for each corresponding step.
        length: A Variable containing a LongTensor of size (batch,)
            which contains the length of each data in a batch.

    Returns:
        loss: An average loss value masked by the length.
    """
    # logits_flat: (batch * max_len, num_classes)
    logits_flat = logits.view(-1, logits.size(-1))
    # log_probs_flat: (batch * max_len, num_classes)
    log_probs_flat = F.log_softmax(logits_flat)
    # target_flat: (batch * max_len, 1)
    target_flat = target.view(-1, 1)
    # losses_flat: (batch * max_len, 1)
    losses_flat = -torch.gather(log_probs_flat, dim=1, index=target_flat)
    # losses: (batch, max_len)
    losses = losses_flat.view(*target.size())
    # mask: (batch, max_len)
    mask = _sequence_mask(sequence_length=length, max_len=target.size(1))
    losses = losses * mask.double()
    loss = losses.sum() / length.double().sum()
    return loss

In [9]:
class MetaLearner(nn.Module):
    
    def __init__(self,
                 input_size,
                 hidden_size):
        super(MetaLearner,self).__init__()
        
    

In [10]:
class Learner(nn.Module):
    
    def __init__(self,
                 input_size,
                 hidden_size,
                 output_size,
                 learning_rate,
                 embeddings=one_hot_embeddings):
        
        super(Learner,self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.encoder = EncoderLSTM(input_size, hidden_size)
        self.decoder = DecoderLSTM(input_size, hidden_size, output_size)
        self.encoder_optimizer = torch.optim.Adam(self.encoder.parameters(), lr=learning_rate)
        self.decoder_optimizer = torch.optim.Adam(self.decoder.parameters(), lr=learning_rate)
        
        self.embeddings = embeddings
        self.criterion = nn.CrossEntropyLoss()
        
    
    def forward(self, sequence, numbered_seq, hidden):
        
        encoder = self.encoder
        decoder = self.decoder
        embeddings = self.embeddings
        criterion = self.criterion
        
        # (seq_length, batch_size, vocab_size)
        seq_size = sequence.size()
        batch_size = seq_size[1]
        sequence_length = seq_size[0]
        loss = 0
        
        # h_n = (1, batch_size, encoding_size)
        encoder_hidden = encoder.initHidden(hidden, batch_size)
        encoder_hidden = (encoder_hidden, encoder_hidden) # Need a tuple

        # Encoder is fed the flipped control sequence
        for index_control in np.arange(sequence_length-1, 0, -1):
            encoder_input = sequence[index_control].unsqueeze(0) # (1, batch_size, vocab_size)
            encoder_hidden = encoder(encoder_input, encoder_hidden)
        
        # feed encoder_hidden
        decoder_input = sequence[1].unsqueeze(0) # One after SOS
        decoder_hidden = encoder_hidden
        predicted_note_index = 0
        
        # Prepare the results tensor
        all_decoder_outputs = Variable(torch.zeros(*sequence.size())).double() # (seq_length, batch_size, vocab_size)
        if use_cuda:
            all_decoder_outputs = all_decoder_outputs.cuda()

        all_decoder_outputs[0] = decoder_input

        for index_control in range(2, sequence_length):
            # decoder_input = decoder_input.view(1, 1, vocabulary_size)
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            
            if random.random() <= 0.9:
                decoder_input = sequence[index_control].unsqueeze(0)
            else:
                topv, topi = decoder_output.data.topk(1)
                # This is the next input, without teacher forcing it's the predicted output
                decoder_input = torch.stack([Variable(torch.DoubleTensor(embeddings[ni]))
                                         for ni in topi.squeeze()]).unsqueeze(0)
                if use_cuda:
                    decoder_input = decoder_input.cuda()
                    
            # Save the decoder output
            all_decoder_outputs[index_control] = decoder_output
        
        
        seq_lens = Variable(torch.LongTensor(np.ones(batch_size, dtype=int)*sequence_length))
        if use_cuda:
            seq_lens = seq_lens.cuda()
        loss = compute_loss(all_decoder_outputs.transpose(0,1).contiguous(),
                        numbered_seq.transpose(0,1).contiguous(), 
                        seq_lens)
        
        return loss
    
    def map_inference(self, sequence, hidden, embeddings=one_hot_embeddings, max_length=500):
        """ sequence has to be batch_size=1"""
        encoder = self.encoder
        decoder = self.decoder
        
        output_control_sequence = []
    
        # Encoder
        encoder_hidden = encoder.initHidden(hidden, batch_size=1)
        encoder_hidden = (encoder_hidden, encoder_hidden)

        sequence_length = sequence.size()[1]

        for index_control in np.arange(sequence_length-1, 0, -1):
            print("Vocab: %d" % vocabulary_size)
            print("Idx: %d" % index_control)
            print(sequence)
            tmp = sequence[0][index_control]
            print(tmp)
            encoder_input = tmp.view(1, 1, vocabulary_size)
            encoder_hidden = encoder(encoder_input, encoder_hidden) # Gets hidden for next input

        # This point we have last encoder_hidden, feed into decoder
        decoder_hidden = encoder_hidden
        decoder_input = sequence[0][0]
        predicted_control_index = SOS_TOKEN

        cur_length = 0
        while True:
            decoder_input = decoder_input.view(1, 1, vocabulary_size)
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)

            # MAP inference
            topv, topi = decoder_output.data.topk(1)
            predicted_control_index = int(topi)
            if predicted_control_index == EOS_TOKEN:
                break
            output_control_sequence.append(predicted_control_index)

            # This is the next input
            decoder_input = torch.from_numpy(embeddings[predicted_control_index])
            decoder_input = Variable(decoder_input).double()
            if use_cuda:
                decoder_input = decoder_input.cuda()

            cur_length += 1
            if cur_length >= max_length:
                break

        return output_control_sequence
    
    def train(self, sequence, numbered_seq, hidden):
        self.encoder_optimizer.zero_grad()
        self.decoder_optimizer.zero_grad()
        
        loss = self.forward(sequence, numbered_seq, hidden)
        
        loss.backward()
        self.encoder_optimizer.step()
        self.decoder_optimizer.step()
        return loss

learner = Learner(vocabulary_size, 
              encoding_size, 
              vocabulary_size,
              learning_rate=0.01)

In [30]:
#input_files = ['bach_846.mid', 'mz_311_1.mid', 'rac_op3_2.mid']
input_files = ['letsgetphysical.mid']
input_variables = []
original_sequences = []

for index, input_file in enumerate(input_files):
    orig_seq = loader.read('../data/' + input_file)
    orig_seq = loader.tokenize(orig_seq)
    
    orig_seq2 = eps.dataset.load('Olivia Newton John', 'letsgetphysical.mid')
    
    trunc_seq = orig_seq[0:500]
    
    print(trunc_seq)
    print(orig_seq2)

    trunc_seq = orig_seq2
 
    trunc_seq = [SOS_TOKEN] + trunc_seq + [EOS_TOKEN]
    original_sequences.append(trunc_seq)
    seq_length = len(trunc_seq)
    
    for n in list(trunc_seq):
        print(n)
    
    trunc_seq = torch.from_numpy(np.array(one_hot_embeddings[list(trunc_seq)])) # This is really time consuming
    trunc_seq = trunc_seq.view(seq_length, vocabulary_size)
    trunc_seq = Variable(trunc_seq)
    if use_cuda:
        trunc_seq = trunc_seq.cuda()
    input_variables.append(trunc_seq)
    
original_sequences = np.array(original_sequences, dtype=np.int64)

Olivia Newton John letsgetphysical.mid
[4248, 557, 4251, 560, 4609, 4281, 673, 4613, 2605, 4610, 2608, 2721, 4625, 4249, 560, 557, 4609, 4284, 673, 4632, 4252, 562, 4249, 559, 4609, 2721, 4250, 579, 4249, 576, 4250, 583, 4281, 675, 4609, 2608, 4609, 2605, 4614, 2627, 579, 4610, 2624, 2627, 4249, 576, 4609, 2631, 4250, 583, 4609, 2624, 2631, 4624, 2610, 4610, 2607, 2723, 4626, 4252, 564, 4609, 4251, 560, 4283, 676, 4631, 2608, 4610, 4248, 579, 4249, 576, 583, 4609, 2724, 4609, 2612, 4625, 2627, 2624, 4248, 579, 4249, 576, 4609, 2631, 2627, 2624, 583, 4610, 2631, 4610, 4252, 561, 4249, 565, 4609, 677, 4633, 2725, 562, 678, 4252, 566, 4609, 2609, 4610, 2613, 4677, 2610, 4611, 578, 4247, 581, 4248, 574, 4613, 2726, 4612, 2614, 4610, 2626, 4252, 578, 4609, 2629, 2622, 4247, 581, 4248, 574, 4609, 2626, 4609, 2629, 2622, 4619, 4245, 564, 4281, 680, 4614, 2612, 4627, 2728, 4253, 567, 4284, 683, 4633, 4251, 578, 4248, 581, 574, 4623, 2626, 4251, 578, 4609, 2629, 2626, 4248, 581, 4609, 2622, 574

IndexError: index 13667 is out of bounds for axis 0 with size 4711

In [14]:
eps.dataset.load('Olivia Newton John', 'letsgetphysical.mid')

Olivia Newton John letsgetphysical.mid


array([4248,  557, 4251,  560, 4609, 4281,  673, 4613, 2605, 4610, 2608,
       2721, 4625, 4249,  560,  557, 4609, 4284,  673, 4632, 4252,  562,
       4249,  559, 4609, 2721, 4250,  579, 4249,  576, 4250,  583, 4281,
        675, 4609, 2608, 4609, 2605, 4614, 2627,  579, 4610, 2624, 2627,
       4249,  576, 4609, 2631, 4250,  583, 4609, 2624, 2631, 4624, 2610,
       4610, 2607, 2723, 4626, 4252,  564, 4609, 4251,  560, 4283,  676,
       4631, 2608, 4610, 4248,  579, 4249,  576,  583, 4609, 2724, 4609,
       2612, 4625, 2627, 2624, 4248,  579, 4249,  576, 4609, 2631, 2627,
       2624,  583, 4610, 2631, 4610, 4252,  561, 4249,  565, 4609,  677,
       4633, 2725,  562,  678, 4252,  566, 4609, 2609, 4610, 2613, 4677,
       2610, 4611,  578, 4247,  581, 4248,  574, 4613, 2726, 4612, 2614,
       4610, 2626, 4252,  578, 4609, 2629, 2622, 4247,  581, 4248,  574,
       4609, 2626, 4609, 2629, 2622, 4619, 4245,  564, 4281,  680, 4614,
       2612, 4627, 2728, 4253,  567, 4284,  683, 46

In [11]:
input_variables = []
original_sequences = []

for index in range(290):
    ep = eps.get_episode()
    orig_seqs = ep.support
    
    for orig_seq in orig_seqs:
        trunc_seq = [SOS_TOKEN] + orig_seq + [EOS_TOKEN]
        original_sequences.append(trunc_seq)
        seq_length = len(trunc_seq)

        trunc_seq = torch.from_numpy(np.array(one_hot_embeddings[trunc_seq])) # This is really time consuming
        trunc_seq = trunc_seq.view(seq_length, vocabulary_size)
        trunc_seq = Variable(trunc_seq)
        if use_cuda:
            trunc_seq = trunc_seq.cuda()
        input_variables.append(trunc_seq)

original_sequences = np.array(original_sequences, dtype=np.int64)

Olivia Newton John helpmemakeitthroughthenight.mid
Olivia Newton John ifyoucouldreadmymind.mid
Olivia Newton John thepromise(thedolphinsong).mid
Olivia Newton John hopelesslydevotedtoyou.mid
Olivia Newton John lookatmeimsandradee.mid
Olivia Newton John xanadu.mid
Olivia Newton John waterunderthebridge.mid
Olivia Newton John magic.mid
Olivia Newton John angelofthemorning.mid
Olivia Newton John longlivelove.mid
Olivia Newton John itssoeasy.mid
Olivia Newton John theairthatibreathe.mid
Olivia Newton John youretheonethatiwant.mid
Olivia Newton John ifweonlyhavelove.mid
Olivia Newton John letsgetphysical.mid


IndexError: index 14127 is out of bounds for axis 0 with size 4711

In [ ]:
""" Testing Learner """
print_every = 10
total_epochs = 200
print_loss_total = 0
batch_size = 2
startTime = time.time()
for epoch in range(1, total_epochs+1):
    for batch in range(len(input_variables)//batch_size):
        # lstm input is (seq_len, batch_size, vocab_size)
        start, end = batch*batch_size, (batch+1)*batch_size
        sequences = torch.stack(input_variables[start:end]).transpose(0,1)
        numbered_seqs = torch.stack(Variable(torch.from_numpy(original_sequences[start:end]))).transpose(0,1)
        if use_cuda:
            numbered_seqs = numbered_seqs.cuda()
        loss = learner.train(sequences, numbered_seqs, hidden=None)
        print_loss_total += loss
    
    if epoch % print_every == 0:
        print_loss_avg = print_loss_total / print_every
        print_loss_total = 0
        print('%s (%d %d%%) %.4f' % (timeSince(startTime, epoch / total_epochs),
                                     epoch, epoch / total_epochs * 100, print_loss_avg))
        

In [ ]:
song_index = 0
test_seq = input_variables[song_index][:,1:101] # First dimension is batch
out_seq = learner.map_inference(test_seq, hidden=None)
whole_seq = original_sequences[song_index][0:100].tolist() + out_seq
midi = loader.detokenize(np.array(whole_seq))
midi.write('test.mid')

In [ ]:
test_hidden = learner.encoder.initHidden(None).squeeze()
print(torch.stack([test_hidden, test_hidden]).unsqueeze(0))